In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split

melb_data = pd.read_csv('melb_data.csv')

y = melb_data.Price
features = ['Rooms', 'Landsize']
X = melb_data[features].copy()

melb_predictors = melb_data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2)


In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [26]:
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid)

np.float64(187701.59979837295)

In [38]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()

imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print('MAE from Approach 2 (Imputation):')
score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid)


MAE from Approach 2 (Imputation):


np.float64(177169.23472017673)

In [41]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
177253.7023073147


In [46]:
X_train.shape

missing_val_count_by_column = (X_train.isnull().sum())

missing_val_count_by_column[missing_val_count_by_column> 0]

Car               48
BuildingArea    5106
YearBuilt       4264
dtype: int64

In [66]:

X_full = pd.read_csv('melb_data.csv')
X_test_full = pd.read_csv('melb_data.csv')

X_full.dropna(axis=0, subset=['Price'], inplace=True)

X_full.shape


(13580, 21)